# Решение

## 1. Скачать файл по ссылке

In [1]:
!wget 'https://raw.githubusercontent.com/mwaskom/seaborn-data/master/titanic.csv' 'titanic.csv'

--2022-02-10 12:29:38--  https://raw.githubusercontent.com/mwaskom/seaborn-data/master/titanic.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.111.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 57018 (56K) [text/plain]
Saving to: ‘titanic.csv’

titanic.csv         100%[===================>]  55.68K  --.-KB/s    in 0.01s   

2022-02-10 12:29:38 (5.08 MB/s) - ‘titanic.csv’ saved [57018/57018]

--2022-02-10 12:29:38--  http://titanic.csv/
Resolving titanic.csv (titanic.csv)... failed: Name or service not known.
wget: unable to resolve host address ‘titanic.csv’
FINISHED --2022-02-10 12:29:38--
Total wall clock time: 0.2s
Downloaded: 1 files, 56K in 0.01s (5.08 MB/s)


## Изучение данных

In [2]:
import pandas as pd

In [15]:
df = pd.read_csv('/content/titanic.csv')

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 15 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   survived     891 non-null    int64  
 1   pclass       891 non-null    int64  
 2   sex          891 non-null    object 
 3   age          714 non-null    float64
 4   sibsp        891 non-null    int64  
 5   parch        891 non-null    int64  
 6   fare         891 non-null    float64
 7   embarked     889 non-null    object 
 8   class        891 non-null    object 
 9   who          891 non-null    object 
 10  adult_male   891 non-null    bool   
 11  deck         203 non-null    object 
 12  embark_town  889 non-null    object 
 13  alive        891 non-null    object 
 14  alone        891 non-null    bool   
dtypes: bool(2), float64(2), int64(4), object(7)
memory usage: 92.4+ KB


### Пропуски:
* возраст (age, 177 пропусков)
* порт посадки (embarked, 2 пропуска)
* палуба (deck, 688 пропусков)
* город посадки (embark_town, 2 пропуска)

### Заполениние пропусков
#### Возраст 
Принято решение написать небольшую модель, обучить ее на заполненных данных и предугадать примерный возраст в пропущенных ячейках. 

#### Порт посадки 
Все пропуски = U (unknown).

#### Палуба
Для определения палубы будет написана другая модель, но ориентироваться на результаты ее работы я не буду, так как тренировочная выборка слишком мала (203 элемента).

#### Город посадки
Все пропуски = Unknown.

#### Пропуски посадки

In [20]:
def fill_na_embark(data_frame: pd.core.frame.DataFrame) -> pd.core.frame.DataFrame: 
    data_frame['embarked'] = data_frame['embarked'].fillna('U')
    data_frame['embark_town'] = data_frame['embark_town'].fillna('Unknown')

    return data_frame

In [32]:
def for_print_embark(data_frame: pd.core.frame.DataFrame, column_offset=4) -> pd.core.frame.DataFrame:
    for_print = [('Порты:', 'Города:')]
    max_len_port = len(for_print[0][0])
    output_str = ''

    unique_ports, unique_towns = df['embarked'].unique(), df['embark_town'].unique()
    
    for index in range(max(len(unique_ports), len(unique_towns))):
        temp_port = unique_ports[index] if index < len(unique_ports) else ''
        temp_town = unique_towns[index] if index < len(unique_towns) else ''

        max_len_port = max(max_len_port, len(temp_port))

        for_print.append((temp_port, temp_town))

    for port, town in for_print: 
        output_str += f'{port}{" " * (max_len_port + column_offset - len(port))}{town}\n'

    return output_str

In [33]:
df = fill_na_embark(df)
print(for_print_embark(df))

Порты:    Города:
S         Southampton
C         Cherbourg
Q         Queenstown
U         Unknown



#### Пропуски возраста

In [34]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense

from tensorflow.keras.optimizers import Adam

from tensorflow.keras.losses import mae
from tensorflow.keras.metrics import mse

In [44]:
def prepare_data(data_frame): 
    data_frame['sex'] = data_frame['sex'].apply(lambda x: 0 if x == 'male' else 1)
    print(data_frame['age'].max())
    data_frame['age'] = data_frame['age'].apply(lambda age: MAX_AGE / age)
    return data_frame

In [46]:
X_COLUMNS = ['pclass', 'sex', 'sibsp', 'parch', 'fare', 'class', 'alive', 'alone']
Y_COLUMNS = ['age']
MAX_AGE = 100

data = df[df['age'].notna()]
prepare_data(df)

238.0952380952381


,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,1,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,1,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,1,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,1,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,1,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,0,2,1,27.0,0,0,13.0000,S,Second,man,True,NaN,Southampton,no,True
887,1,1,1,19.0,0,0,30.0000,S,First,woman,False,B,Southampton,yes,True
888,0,3,1,NaN,1,2,23.4500,S,Third,woman,False,NaN,Southampton,no,False
889,1,1,1,26.0,0,0,30.0000,C,First,man,True,C,Cherbourg,yes,True
